In [1]:
!pip list

Package                                  Version
---------------------------------------- -----------
Adafruit-Blinka                          8.66.0
Adafruit-Blinka-Raspberry-Pi5-Neopixel   1.0.0rc2
adafruit-circuitpython-busdevice         5.2.13
adafruit-circuitpython-connectionmanager 3.1.5
adafruit-circuitpython-motor             3.4.17
adafruit-circuitpython-pca9685           3.4.19
adafruit-circuitpython-register          1.10.4
adafruit-circuitpython-requests          4.1.13
adafruit-circuitpython-servokit          1.3.21
adafruit-circuitpython-typing            1.12.2
Adafruit-PlatformDetect                  3.83.1
Adafruit-PureIO                          1.1.11
asttokens                                3.0.0
binho-host-adapter                       0.1.6
colorzero                                2.0
comm                                     0.2.3
debugpy                                  1.8.16
decorator                                5.2.1
executing                               

In [ ]:
!pip install opencv-python

In [1]:
import cv2
import time

# Open camera
cap = cv2.VideoCapture("/dev/video0")  # use /dev/video1 if needed
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Grab one frame
ret, frame = cap.read()
cap.release()

if not ret:
    print("❌ Failed to grab image")
    exit()


cv2.imshow("Captured", frame)
cv2.waitKey(2000)  # display for 2 seconds
cv2.destroyAllWindows()

gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
avg_brightness = gray.mean()
print(avg_brightness)


139.71228841145833


In [4]:
import cv2
import numpy as np

# --- Real-world size of cube (in inches) ---
KNOWN_WIDTH_INCH = 1.0

# --- Distance (inches) at which you take a reference photo ---
KNOWN_DISTANCE_INCH = 12.0  # 1 foot away

# --- Color range for your cube (example: red cube) ---
# You can change these for blue, green, etc.
LOWER_COLOR = np.array([0, 120, 70])      # HSV lower bound
UPPER_COLOR = np.array([10, 255, 255])    # HSV upper bound

# If your red hue wraps around 180, you can also add this range:
LOWER_COLOR_2 = np.array([170, 120, 70])
UPPER_COLOR_2 = np.array([180, 255, 255])

# --- Helper function to find object width in pixels ---
def find_cube_width(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask1 = cv2.inRange(hsv, LOWER_COLOR, UPPER_COLOR)
    mask2 = cv2.inRange(hsv, LOWER_COLOR_2, UPPER_COLOR_2)
    mask = cv2.bitwise_or(mask1, mask2)

    # Clean up the mask
    kernel = np.ones((5,5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return None, mask

    # Take the largest contour (likely the cube)
    c = max(contours, key=cv2.contourArea)
    (x, y, w, h) = cv2.boundingRect(c)
    return (x, y, w, h), mask


# --- STEP 1: Calibrate focal length using a reference photo ---
def calibrate_focal_length():
    print("📸 Calibration: Place cube exactly", KNOWN_DISTANCE_INCH, "inches away and press [SPACE]")
    cap = cv2.VideoCapture(0)
    focal_length = None

    while True:
        ret, frame = cap.read()
        if not ret:
            continue

        cube, mask = find_cube_width(frame)
        if cube:
            (x, y, w, h) = cube
            cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
            cv2.putText(frame, f"Width: {w}px", (x, y-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,255), 2)

        cv2.imshow("Calibration", frame)
        if cv2.waitKey(1) & 0xFF == 32:  # SPACE key
            if cube:
                focal_length = (w * KNOWN_DISTANCE_INCH) / KNOWN_WIDTH_INCH
                print(f"✅ Focal length = {focal_length:.2f} pixels")
            break
        elif cv2.waitKey(1) & 0xFF == 27:  # ESC
            break

    cap.release()
    cv2.destroyAllWindows()
    return focal_length


# --- STEP 2: Estimate distance in live video ---
def estimate_distance(focal_length):
    print("🎥 Estimating distances. Press ESC to quit.")
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            continue

        cube, mask = find_cube_width(frame)
        if cube:
            (x, y, w, h) = cube
            distance_inch = (KNOWN_WIDTH_INCH * focal_length) / w
            cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
            cv2.putText(frame, f"{distance_inch:.2f} in", (x, y-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,255), 2)

        cv2.imshow("Distance Estimation", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    focal_length = calibrate_focal_length()
    if focal_length:
        estimate_distance(focal_length)
    else:
        print("❌ Calibration failed. Try again.")


📸 Calibration: Place cube exactly 12.0 inches away and press [SPACE]


KeyboardInterrupt: 

In [5]:
!ls

trained_yolo_model  vision.ipynb
